Импортируем необходимые библиотеки и считываем необходимую таблицу в переменную *table* 

In [1]:
import pandas as pd
import gspread
import numpy as np
from gspread_dataframe import set_with_dataframe
gc = gspread.service_account(filename= None) #здесь необходимо ввести путь и имя сгенерированного ключа на Google Cloud Platform
#для доступа к таблицам Google Sheets
table = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ/edit#gid=0')

Преобразуем вкладки таблицы в pandas Dataframe для удобства анализа данных, а также преобразуем столбец **created_at** в тип *datetime64* для удобства в вычислениях

In [2]:
df_leads = pd.DataFrame(table.worksheet('leads').get_all_records(default_blank=np.nan))
df_managers = pd.DataFrame(table.worksheet('managers').get_all_records(default_blank=np.nan))
df_transactions = pd.DataFrame(table.worksheet('transactions').get_all_records(default_blank=np.nan))
df_clients = pd.DataFrame(table.worksheet('clients').get_all_records(default_blank=np.nan))

df_leads['created_at'] = df_leads['created_at'].astype('datetime64')
df_transactions['created_at'] = df_transactions['created_at'].astype('datetime64')

Посмотрим, какие значения принимает столбец **d_utm_source** вкладки *leads*

In [3]:
df_leads['d_utm_source'].value_counts()

instagram                                   920
utm_source                                  379
viber                                       205
yandex                                      164
google                                      145
vk                                           98
sms                                          83
vkontakte                                    61
insta                                        19
ycard                                        15
ig                                            4
ycard#!/tproduct/225696739-1498486363994      1
Name: d_utm_source, dtype: int64

Видим, что каналы привлечения дублируются, уницифицируем их для получения объективного отчета по данным.  
Также заменим неизвестные значения данного столбца (*d_utm_source*) на 'unknown', чтобы данные лиды тоже учитывать в pivot_table


In [4]:
df_leads['d_utm_source'].replace({'vk':'vkontakte','insta':'instagram','ig':'instagram',
                                  'ycard#!/tproduct/225696739-1498486363994':'ycard'}, inplace = True)
df_leads['d_utm_source'].fillna('unknown', inplace=True)
df_leads['d_utm_source'].value_counts()

unknown       1243
instagram      943
utm_source     379
viber          205
yandex         164
vkontakte      159
google         145
sms             83
ycard           16
Name: d_utm_source, dtype: int64

**Введите временной период**

Введите временной период для составления отчета в виде полуинтервала [ )

In [5]:
dt_period=['2020-10-15','2020-10-30']

In [6]:
df_leads_period = df_leads.query('@dt_period[0] <= created_at <= @dt_period[1]')

In [7]:
print(df_leads_period.columns)
print(df_managers.columns)

Index(['lead_id', 'created_at', 'd_utm_medium', 'd_utm_source', 'l_manager_id',
       'l_client_id'],
      dtype='object')
Index(['manager_id', 'd_manager', 'd_club'], dtype='object')


In [8]:
df = df_leads_period.merge(df_managers, how='left', left_on = 'l_manager_id', right_on = 'manager_id')\
.drop('manager_id',axis=1)
df

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,d_manager,d_club
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1
2,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vkontakte,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,manager #10,club #2
3,00aea163-13a5-11eb-ac1d-c412f533dba1,2020-10-21 17:54:59,NaN,unknown,db06b8bc-bfb2-11e7-8116-c412f533dba1,00aea164-13a5-11eb-ac1d-c412f533dba1,manager #4,club #2
4,01769e26-19dd-11eb-ac1d-c412f533dba1,2020-10-29 15:51:05,cpc,instagram,ad52c7a8-a752-11e7-8115-c412f533dba1,00000000-0000-0000-0000-000000000000,manager #7,club #1
...,...,...,...,...,...,...,...,...
808,ff466f12-1555-11eb-ac1d-c412f533dba1,2020-10-23 21:34:27,cpc,instagram,9a87c608-52dc-11ea-abeb-c412f533dba1,00000000-0000-0000-0000-000000000000,manager #9,club #3
809,ff7d0045-12c7-11eb-ac1c-c412f533dba1,2020-10-20 15:32:56,sms,viber,9a87c608-52dc-11ea-abeb-c412f533dba1,2e4fbe67-12cb-11eb-ac1c-c412f533dba1,manager #9,club #3
810,ffaa0815-16c4-11eb-ac1d-c412f533dba1,2020-10-25 17:21:33,NaN,unknown,5dd88f32-6501-11ea-abf3-c412f533dba1,ffaa0816-16c4-11eb-ac1d-c412f533dba1,manager #10,club #2
811,ffacb5af-194f-11eb-ac1d-c412f533dba1,2020-10-28 23:01:35,cpc,vkontakte,ad52c7a8-a752-11e7-8115-c412f533dba1,377ea5e0-d089-11e9-abbd-c412f533dba1,manager #7,club #1


**Количество заявок**

In [9]:
df_count_pivot = df.pivot_table(values = 'lead_id', index = ['d_club','d_manager'], columns='d_utm_source', 
                               aggfunc = 'count', fill_value = 0, margins = True)
df_count_pivot

d_utm_source         google  instagram  unknown  utm_source  viber  vkontakte  \
d_club  d_manager                                                               
club #1 manager #1        7         43       16          10     10          2   
        manager #7        3         34       25           7      4          5   
club #2 manager #10       2         10       19           5      3         11   
        manager #4        4         18       22          13     10          6   
        manager #8        2         17       16           5      8          0   
club #3 manager #13       1          1        3           0      1          0   
        manager #9       11         58       42          13     25          8   
club #4 manager #11       9         24       32           4      3          8   
        manager #2        1         20        9           5      4          4   
        manager #5        4         25       28           7      9          5   
All                      44        250      212          69     77         49   

d_utm_source         yandex  All  
d_club  d_manager                 
club #1 manager #1        4   92  
        manager #7        3   81  
club #2 manager #10       3   53  
        manager #4        4   77  
        manager #8        3   51  
club #3 manager #13       0    6  
        manager #9       14  171  
club #4 manager #11       3   83  
        manager #2        3   46  
        manager #5        0   78  
All                      37  738

**Количество мусорных заявок**

In [10]:
df_garbage = df.query('l_client_id == "00000000-0000-0000-0000-000000000000"')
df_garbage_pivot = df_garbage.pivot_table(values = 'lead_id', index = ['d_club','d_manager'], columns='d_utm_source', 
                               aggfunc = 'count', fill_value = 0, margins = True)
df_garbage_pivot.to_excel('result.xlsx', sheet_name='Количество мусорных заявок')
df_garbage_pivot

d_utm_source         google  instagram  unknown  utm_source  viber  vkontakte  \
d_club  d_manager                                                               
club #1 manager #1        3         11        5           2      2          0   
        manager #7        1          5        4           0      0          0   
club #2 manager #10       0          0        1           0      0          0   
        manager #8        0          1        1           0      0          0   
club #3 manager #13       1          0        0           0      0          0   
        manager #9        1          3        0           0      1          0   
club #4 manager #11       2          3       22           0      0          2   
        manager #2        0          0        4           0      0          0   
        manager #5        1          1       10           0      0          0   
All                       9         24       47           2      3          2   

d_utm_source         yandex  All  
d_club  d_manager                 
club #1 manager #1        1   24  
        manager #7        0   10  
club #2 manager #10       0    1  
        manager #8        0    2  
club #3 manager #13       0    1  
        manager #9        0    5  
club #4 manager #11       0   29  
        manager #2        0    4  
        manager #5        0   12  
All                       1   88

**Количество новых заявок**

In [11]:
df_without_garbage = df.query('l_client_id != "00000000-0000-0000-0000-000000000000"')
clients = set(df_without_garbage['l_client_id'])
old_clients = set(pd.concat([df_leads.query('created_at < @dt_period[0]')['l_client_id'], 
                            df_transactions.query('created_at < @dt_period[0]')['l_client_id']]))
new_clients = set()
for client in clients:
    if client not in old_clients:
        new_clients.add(client)
df_new_leads = df.query('l_client_id in @new_clients')
df_new_leads_pivot = df_new_leads.pivot_table(values = 'lead_id', index = ['d_club','d_manager'], 
                                              columns='d_utm_source',aggfunc = 'count', fill_value = 0, margins = True)
df_new_leads_pivot

d_utm_source         google  instagram  unknown  utm_source  viber  vkontakte  \
d_club  d_manager                                                               
club #1 manager #1        4         29        9           8      7          1   
        manager #7        2         29       21           7      3          5   
club #2 manager #10       1          8       16           4      1          9   
        manager #4        3         14       22          13      7          4   
        manager #8        2         15       15           5      4          0   
club #3 manager #13       0          1        2           0      1          0   
        manager #9        6         48       39          13     17          5   
club #4 manager #11       5         18        8           4      2          5   
        manager #2        1         17        5           5      3          3   
        manager #5        3         22       17           7      8          5   
All                      27        201      154          66     53         37   

d_utm_source         yandex  All  
d_club  d_manager                 
club #1 manager #1        2   60  
        manager #7        3   70  
club #2 manager #10       3   42  
        manager #4        3   66  
        manager #8        3   44  
club #3 manager #13       0    4  
        manager #9       13  141  
club #4 manager #11       1   43  
        manager #2        3   37  
        manager #5        0   62  
All                      31  569

**Количество покупателей**

In [12]:
df_leads_trans = df_without_garbage.merge(df_transactions, how='left', left_on = 'l_client_id', 
                                          right_on = 'l_client_id')
df_leads_trans.rename({'created_at_x':'lead_created_at','created_at_y':'trans_created_at'},axis=1, inplace=True)
df_leads_trans['lead_created_at+8'] = df_leads_trans['lead_created_at'] + pd.to_timedelta(8,unit='D')
df_leads_trans.head(5)

,lead_id,lead_created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,d_manager,d_club,transaction_id,trans_created_at,m_real_amount,lead_created_at+8
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,3519d02c-9753-40fb-9135-cd769c3173b1,2020-11-26 18:38:31,563.0,2020-10-24 18:08:09
1,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,3519d02c-9753-40fb-9135-cd769c3173b1,2020-11-26 18:38:31,0.0,2020-10-24 18:08:09
2,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,3519d02c-9753-40fb-9135-cd769c3173b1,2020-11-26 18:38:31,3.0,2020-10-24 18:08:09
3,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,0b51d2b8-61b4-4421-a23d-1a1f4998d425,2020-10-22 16:42:29,44041.0,2020-10-24 18:08:09
4,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1,NaN,NaT,NaN,2020-10-23 23:45:40


In [13]:
df_buyers = df_leads_trans[(df_leads_trans['lead_created_at'] <= df_leads_trans['trans_created_at']) & 
                          (df_leads_trans['trans_created_at'] < df_leads_trans['lead_created_at+8'])]

In [14]:
df_buyers.head(5)

,lead_id,lead_created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,d_manager,d_club,transaction_id,trans_created_at,m_real_amount,lead_created_at+8
3,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,0b51d2b8-61b4-4421-a23d-1a1f4998d425,2020-10-22 16:42:29,44041.0,2020-10-24 18:08:09
9,018378bf-1796-11eb-ac1d-c412f533dba1,2020-10-26 18:17:41,NaN,unknown,43756fa4-57a0-11e9-ab9a-c412f533dba1,1d7e0076-1792-11eb-ac1d-c412f533dba1,manager #11,club #4,d451bf89-a236-444e-92dc-2beee3980047,2020-10-29 17:41:18,40739.0,2020-11-03 18:17:41
14,028dc5b3-1852-11eb-ac1d-c412f533dba1,2020-10-27 16:43:28,cpc,instagram,1e9c5562-0cfc-11ea-abe1-c412f533dba1,50010dbc-03c2-11ea-abdf-c412f533dba1,manager #8,club #2,1d476a5e-98e3-40bb-b42a-11119e506da2,2020-10-28 12:44:21,37698.0,2020-11-04 16:43:28
38,03e81b35-161d-11eb-ac1d-c412f533dba1,2020-10-24 21:19:04,utm_medium,utm_source,d12fe765-b444-11e9-abb3-c412f533dba1,16433eb5-dc11-11e7-8122-c412f533dba1,manager #1,club #1,533bf8b0-819b-448b-9357-94cbc6e4498c,2020-10-25 10:47:10,61443.0,2020-11-01 21:19:04
39,03e81b35-161d-11eb-ac1d-c412f533dba1,2020-10-24 21:19:04,utm_medium,utm_source,d12fe765-b444-11e9-abb3-c412f533dba1,16433eb5-dc11-11e7-8122-c412f533dba1,manager #1,club #1,ef65a399-9532-49c3-abd0-4f43d28d690e,2020-10-30 12:16:15,2137.0,2020-11-01 21:19:04


In [15]:
df_buyers_pivot = df_buyers.pivot_table(values = 'lead_id', index = ['d_club','d_manager'], 
                                              columns='d_utm_source',aggfunc = 'count', fill_value = 0, margins = True)
df_buyers_pivot

d_utm_source         google  instagram  unknown  utm_source  viber  vkontakte  \
d_club  d_manager                                                               
club #1 manager #1        2          4        8           4      2          2   
        manager #7        0          3        4           4      1          3   
club #2 manager #10       0          1        6           3      1          5   
        manager #4        2          2        5           4      1          2   
        manager #8        2          4        1           1      3          0   
club #3 manager #9        1          8        8           2      4          4   
club #4 manager #11       2          2        5           1      0          2   
        manager #2        1          4        1           3      0          1   
        manager #5        0          1        2           1      4          4   
All                      10         29       40          23     16         23   

d_utm_source         yandex  All  
d_club  d_manager                 
club #1 manager #1        0   22  
        manager #7        0   15  
club #2 manager #10       4   20  
        manager #4        2   18  
        manager #8        3   14  
club #3 manager #9        2   29  
club #4 manager #11       0   12  
        manager #2        1   11  
        manager #5        0   12  
All                      12  153

**Количество новых покупателей**

In [16]:
buyer_clients = set(df_buyers['l_client_id'])
old_buyer_clients = set(df_transactions.query('created_at < @dt_period[0]')['l_client_id'])
new_buyer_clients = set()
for client in buyer_clients:
    if client not in old_buyer_clients:
        new_buyer_clients.add(client)
df_new_buyers = df.query('l_client_id in @new_buyer_clients')
df_new_buyers_pivot = df_new_buyers.pivot_table(values = 'lead_id', index = ['d_club','d_manager'], 
                                              columns='d_utm_source',aggfunc = 'count', fill_value = 0, margins = True)
df_new_buyers_pivot

d_utm_source         google  instagram  unknown  utm_source  viber  vkontakte  \
d_club  d_manager                                                               
club #1 manager #1        2          3        4           3      2          1   
        manager #7        0          3        3           3      0          3   
club #2 manager #10       0          0        4           0      1          5   
        manager #4        2          1        4           3      1          2   
        manager #8        2          4        1           1      2          0   
club #3 manager #9        1          7        8           2      2          4   
club #4 manager #11       2          2        2           1      0          2   
        manager #2        1          2        1           3      0          0   
        manager #5        0          1        2           1      2          4   
All                      10         23       29          17     10         21   

d_utm_source         yandex  All  
d_club  d_manager                 
club #1 manager #1        0   15  
        manager #7        0   12  
club #2 manager #10       2   12  
        manager #4        2   15  
        manager #8        3   13  
club #3 manager #9        1   25  
club #4 manager #11       0    9  
        manager #2        1    8  
        manager #5        0   10  
All                       9  119

**Доход от покупок новых покупателей**

In [17]:
df_new_buyers.head(1)

,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,d_manager,d_club
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2


In [18]:
df_new_buyers['income'] = df_new_buyers['l_client_id'].apply(lambda x: sum(df_transactions[df_transactions['l_client_id'] == x]
                                                                           ['m_real_amount']))
df_new_buyers.head(5)

<ipython-input-18-575d4f0bdb45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_buyers['income'] = df_new_buyers['l_client_id'].apply(lambda x: sum(df_transactions[df_transactions['l_client_id'] == x]


,lead_id,created_at,d_utm_medium,d_utm_source,l_manager_id,l_client_id,d_manager,d_club,income
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,44607
5,018378bf-1796-11eb-ac1d-c412f533dba1,2020-10-26 18:17:41,NaN,unknown,43756fa4-57a0-11e9-ab9a-c412f533dba1,1d7e0076-1792-11eb-ac1d-c412f533dba1,manager #11,club #4,40924
10,028dc5b3-1852-11eb-ac1d-c412f533dba1,2020-10-27 16:43:28,cpc,instagram,1e9c5562-0cfc-11ea-abe1-c412f533dba1,50010dbc-03c2-11ea-abdf-c412f533dba1,manager #8,club #2,66402
19,03e81b35-161d-11eb-ac1d-c412f533dba1,2020-10-24 21:19:04,utm_medium,utm_source,d12fe765-b444-11e9-abb3-c412f533dba1,16433eb5-dc11-11e7-8122-c412f533dba1,manager #1,club #1,63580
21,04141587-1778-11eb-ac1d-c412f533dba1,2020-10-26 14:43:00,NaN,unknown,9a87c608-52dc-11ea-abeb-c412f533dba1,1a32cba7-d5a2-11e9-abbe-c412f533dba1,manager #9,club #3,54430


In [19]:
df_new_buyers_income_pivot = df_new_buyers.pivot_table(values = 'income', index = ['d_club','d_manager'], 
                                              columns='d_utm_source',aggfunc = 'sum', fill_value = 0, margins = True)
df_new_buyers_income_pivot

d_utm_source         google  instagram  unknown  utm_source   viber  \
d_club  d_manager                                                     
club #1 manager #1    62090     147658   178803      130822   87507   
        manager #7        0      56156   104318       65212       0   
club #2 manager #10       0          0   167704           0   18527   
        manager #4   100868      35086   139828      213339   62083   
        manager #8   217806     213529    50815       21083   67211   
club #3 manager #9    40924     267955   360855       91794  124745   
club #4 manager #11   26475      56877    53309       62341       0   
        manager #2    76873      50830    46474      141451       0   
        manager #5        0      42474    63539       36629  113873   
All                  525036     870565  1165645      762671  473946   

d_utm_source         vkontakte  yandex      All  
d_club  d_manager                                
club #1 manager #1       50482       0   657362  
        manager #7       35486       0   261172  
club #2 manager #10     149977   98412   434620  
        manager #4      136540  106578   794322  
        manager #8           0   99769   670213  
club #3 manager #9      201970   56094  1144337  
club #4 manager #11     121409       0   320411  
        manager #2           0   53713   369341  
        manager #5      144405       0   400920  
All                     840269  414566  5052698

Сохраним полученные pivot таблицы в *Google Sheets*

In [20]:
sh = gc.create('result')
sh.share(None,perm_type='anyone', role='reader')

worksheet = sh.add_worksheet(title="Количество заявок",rows=100,cols=20)
set_with_dataframe(worksheet, df_count_pivot, include_index=True)

worksheet = sh.add_worksheet(title="Количество мусорных заявок",rows=100,cols=20)
set_with_dataframe(worksheet, df_garbage_pivot, include_index=True)

worksheet = sh.add_worksheet(title="Количество новых заявок",rows=100,cols=20)
set_with_dataframe(worksheet, df_new_leads_pivot, include_index=True)

worksheet = sh.add_worksheet(title="Количество покупателей",rows=100,cols=20)
set_with_dataframe(worksheet, df_buyers_pivot, include_index=True)

worksheet = sh.add_worksheet(title="Количество новых покупателей",rows=100,cols=20)
set_with_dataframe(worksheet, df_new_buyers_pivot, include_index=True)

worksheet = sh.add_worksheet(title="Доход от покупок новых покупателей",rows=100,cols=20)
set_with_dataframe(worksheet, df_new_buyers_income_pivot, include_index=True)


In [21]:
sh.del_worksheet(sh.worksheets()[0])
sh.url

'https://docs.google.com/spreadsheets/d/1VAr4ALFvylrHxA3Q6tURY0u5I_lWHOJ1fPomjN0A5N8'